In [14]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings
from chains import XRPQuerier, XRPProcessor, XRPPipeline
import logging
from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations

logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

from decoder import LogDecoder
from config import Settings
from web3 import Web3



In [17]:
def decode_hex(value):
    """
    Decode a hexadecimal string to an integer if it's an Ethereum-style integer (e.g., block numbers, gas values).
    Does not decode long hashes or other non-integer hex values.
    :param value: Hexadecimal string (e.g., '0x677df92f') or other types.
    :return: Decoded integer or original value if not a valid short hex integer.
    """
    if isinstance(value, str) and value.startswith("0x"):
        if value == '0x':
            return None
        
        # Only decode if the hex string is short (e.g., block numbers, gas, timestamps)
        return int(value, 16)
    return value  # Return original value if not a short hex integer

# Parallelization of the tasks
def reconcile_logs_with_transactions(block):
    
    transactions = block["transactions"]
    w3 = Web3(Web3.HTTPProvider(Settings.ETHEREUM_ENDPOINT))
    
    # Fetch logs for the block
    logs = w3.eth.get_logs({
        "fromBlock": decode_hex(block['number']),
        "toBlock": decode_hex(block['number'])
    })

    # Group logs by transaction hash
    logs_by_transaction = {}
    for log in logs:
        tx_hash = log["transactionHash"].to_0x_hex()
        if tx_hash not in logs_by_transaction:
            logs_by_transaction[tx_hash] = []
        logs_by_transaction[tx_hash].append(dict(log))

    # Analyze the results
    tx_with_logs = len(logs_by_transaction)
    tx_without_logs = len(transactions) - tx_with_logs

    
    
    print(f"Total transactions in block: {len(transactions)}")
    print(f"Transactions with logs: {tx_with_logs}")
    print(f"Transactions without logs: {tx_without_logs}")

    # Return grouped logs for further analysis
    return logs_by_transaction



In [22]:
mongodb = MongoDatabase()
mongodb_query_ops = MongoQueryOperations(mongodb)

queries = mongodb_query_ops.get_recent_blocks('Ethereum', 100)

query = mongodb_query_ops.get_block_by_number('Ethereum', 21585277)
data = query['raw_block_data']


2025-01-11 02:09:48,562 - INFO - Connected to MongoDB database: blockchain
2025-01-11 02:09:48,592 - INFO - Indexes created for collection: Solana
2025-01-11 02:09:48,593 - INFO - Indexes created for collection: Bitcoin
2025-01-11 02:09:48,594 - INFO - Indexes created for collection: Ethereum
2025-01-11 02:09:48,595 - INFO - Indexes created for collection: BNB
2025-01-11 02:09:48,596 - INFO - Indexes created for collection: XRP
2025-01-11 02:09:48,606 - INFO - Retrieved 100 most recent blocks from the Ethereum collection.
2025-01-11 02:09:48,732 - INFO - Retrieved block 21585277 from Ethereum collection in MongoDB.


In [33]:
logs_by_transaction = reconcile_logs_with_transactions(queries[90]['raw_block_data'])

Total transactions in block: 94
Transactions with logs: 64
Transactions without logs: 30


In [34]:
logs_by_transaction

{'0x446f6ce6b070e9247d7db2f8fdb4aacc0505804588c215f5a00d8babac10133d': [{'address': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x00000000000000000000000088e6a0c2ddd26feeb64f039a2c41296fcb3f5640'),
    HexBytes('0x00000000000000000000000068d3a973e7272eb388022a5c6518d9b2a2e66fbf')],
   'data': HexBytes('0x000000000000000000000000000000000000000000000000e614114686f940eb'),
   'blockNumber': 21598957,
   'transactionHash': HexBytes('0x446f6ce6b070e9247d7db2f8fdb4aacc0505804588c215f5a00d8babac10133d'),
   'transactionIndex': 0,
   'blockHash': HexBytes('0x113d8151dafd700160340275c01aa99d87d180478252f4578287deeb4c578590'),
   'logIndex': 0,
   'removed': False},
  {'address': '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x00000000000000000000000068d3a973e7272eb388022a5c6518d9

In [26]:
log_decoder = LogDecoder(Settings.ETHERSCAN_API_KEY)

results = log_decoder.process_transaction_logs(logs_by_transaction)

Contract: 0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640, Event: c42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67, Count: 1
Contract: 0x1ffEc7119e315B15852557f654AE0052f76e6ae1, Event: 1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1, Count: 1
Contract: 0xD37BbE5744D730a1d98d8DC97c42F0Ca46aD7146, Event: a9cd03aa3c1b4515114539cd53d22085129d495cb9e9f9af77864526240f1bf7, Count: 1
Contract: 0x000000000022D473030F116dDEE9F6B43aC78BA3, Event: c6a377bfc4eb120024a8ac08eef205be16b817020812c73223e81d1bdb9708ec, Count: 1
Contract: 0xD7E4b67E735733aC98a88F13d087D8aac670E644, Event: e3407208b14fa025330ca187030f118a1c0cdb604aba93ba45c862e6095aee27, Count: 2
Contract: 0xDef1C0ded9bec7F1a1670819833240f027b25EfF, Event: 829fa99d94dc4636925b38632e625736a614c154d55006b7ab6bea979c210c32, Count: 1
Contract: 0x5c7BCd6E7De5423a257D81B442095A1a6ced35C5, Event: a123dc29aebf7d0c3322c8eeb5b999e859f39937950ed31056532713d0de396f, Count: 2
Contract: 0x3a23F943181408EAC424116Af7b7790c94Cb

In [30]:
results = []
for i, item in enumerate(queries[10:20]):
    print(f"Processing block {i+10} of {len(queries)}")
    logs_by_transaction = reconcile_logs_with_transactions(item['raw_block_data'])
    results = log_decoder.process_transaction_logs(logs_by_transaction)
    results.append(results)

print(log_decoder.known_events)

Processing block 10 of 100
Total transactions in block: 158
Transactions with logs: 101
Transactions without logs: 57


InvalidPointer: Invalid pointer in tuple at location 0 in payload

In [32]:
logs_by_transaction


{'0xc78e3ce5733e7891418a4650fbd7ea1c68dab38b5f76371dcbe75b8abe506e0c': [{'address': '0xE620F9d552fD3b36AFB0Ef1d2B867b501Dc6ad9F',
   'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'),
    HexBytes('0x000000000000000000000000bbb471a4b5b62912bab7bf8d39b93ec99382af8c'),
    HexBytes('0x000000000000000000000000adf3f407805e5a901531feef8799b6e986099679')],
   'data': HexBytes('0x00000000000000000000000000000000000000000000387b64ccd89a75b5dfff'),
   'blockNumber': 21591791,
   'transactionHash': HexBytes('0xc78e3ce5733e7891418a4650fbd7ea1c68dab38b5f76371dcbe75b8abe506e0c'),
   'transactionIndex': 0,
   'blockHash': HexBytes('0xaf0dad12c1acf901d7de4b2f60b7980777f348743e35de6a91f0a50c81f7ad4d'),
   'logIndex': 0,
   'removed': False},
  {'address': '0xE620F9d552fD3b36AFB0Ef1d2B867b501Dc6ad9F',
   'topics': [HexBytes('0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'),
    HexBytes('0x000000000000000000000000bbb471a4b5b62912bab7bf8d39b93e

In [28]:
log_decoder.known_events

{'ddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef': 'Transfer(address,address,uint256)',
 'c42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67': 'Swap(address,address,int256,int256,uint160,uint128,int24)',
 'd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822': 'Swap(address,uint256,uint256,uint256,uint256,address)',
 '48e9e2c732ba278de6ac88a3a57a5c5ba13d3d8370e709b3b98333a57876ca95': 'NewReport(bytes32,uint256,bytes,uint256,bytes,address)',
 'e1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c': 'Deposit(address,uint256)',
 '8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925': 'Approval(address,address,uint256)',
 '5fe47ed6d4225326d3303476197d782ded5a4e9c14f479dc9ec4992af4e85d59': 'Deposit(address,address,address,uint256,uint256)',
 '1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1': 'Sync(uint112,uint112)',
 '7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65': 'Withdrawal(address,uint